## Chat GPT 4 Build

In [195]:
import requests
from bs4 import BeautifulSoup
import tkinter as tk
from tkinter import StringVar, ttk
from functools import partial
from tkinter import ttk, Text, Scrollbar
from collections import Counter, defaultdict
from functools import partial

### Import Word Lists

In [196]:
def fetch_gist_content(gist_url):
    # Extract the gist ID from the URL
    gist_id = gist_url.split('/')[-1].split('.')[0]
    
    # Construct the URL for the raw content
    raw_url = f"https://gist.githubusercontent.com/dracos/{gist_id}/raw/"
    
    response = requests.get(raw_url)
    
    if response.status_code == 200:
        return string_to_list(response.text)
    else:
        return None

def string_to_list(input_string):
    # Split the string by newline to get a list of words and remove any leading/trailing whitespace
    return [word.upper() for word in input_string.strip().split("\n")]

def get_old_words():
    url = "https://www.rockpapershotgun.com/wordle-past-answers"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")

    oldwords = []
    for li in soup.select("ul.inline li"):
        oldwords.append(li.text)
    return oldwords

def get_frequency_list():
    # Define the file name
    file_name = "five_letter_frequency_list.txt"

    # Initialize an empty dictionary
    word_frequency_dict = {}

    # Read the contents of the file and parse into a dictionary
    try:
        with open(file_name, "r") as file:
            lines = file.readlines()
            for line in lines:
                word, number = line.strip().split('\t')
                word_frequency_dict[word] = int(number)
    except FileNotFoundError:
        print(f"File '{file_name}' not found.")

    return word_frequency_dict

def remove_words(five_letter_words, OldWords):
    """Removes all words from `five_letter_words` that are also in `OldWords`."""
    return [word for word in five_letter_words if word not in OldWords]

def keep_old_words (word_list):
    print("Len word_list ", len(word_list))
    return word_list

### Starting word

In [197]:
def count_letter_frequency(words_list):
    """Counts the frequency of each letter in the given list of words."""
    letter_frequency = Counter("".join(words_list))
    return sorted(letter_frequency.items(), key=lambda x: x[1], reverse=True)

def filter_words_based_on_letters(words_list, top_common_letters, num_letters=10):
    """Filters the words based on the most common letters and extracts unique letters."""
    filtered_words = words_list.copy()
    last_non_empty_list = filtered_words  # Start with the original list as the last non-empty list

    for letter, _ in top_common_letters[:num_letters]:
        temp_filtered = [word for word in filtered_words if letter in word]
        if temp_filtered:  # Check if the list is not empty
            last_non_empty_list = temp_filtered
        filtered_words = temp_filtered

    unique_letters = set("".join(last_non_empty_list))
    return last_non_empty_list, list(unique_letters)

def get_most_common_letter_positions(words_list, letters):
    """Returns the most common positions of the given letters in the word list."""
    letter_positions = {letter: [] for letter in letters}

    for word in words_list:
        for idx, char in enumerate(word):
            if char in letters:
                letter_positions[char].append(idx)

    most_common_positions = {}
    for letter, positions in letter_positions.items():
        if positions:
            most_common_positions[letter] = max(set(positions), key=positions.count)
    
    # Order the positions by their frequencies in descending order
    sorted_positions = sorted(most_common_positions.items(), key=lambda x: x[1], reverse=False)
    
    return sorted_positions

def calculate_word_scores(word_frequency_dict, letter_frequency_in_word_frequency):
    # Assign scores to letters, starting from 1 for the least frequent letter
    letter_scores = {letter: idx+1 for idx, letter in enumerate(reversed(letter_frequency_in_word_frequency))}
    
    # Calculate the score for each word
    word_scores = {}
    for word in word_frequency_dict:
        score = sum([letter_scores[letter] for letter in word])
        word_scores[word] = score
    
    return word_scores


### Filter Out words based on GUI feedback

In [198]:
def filter_words_by_locked_positions(words_list, locked_positions):
    """Filters words that have the locked letters in the locked positions."""
    for position, letter in locked_positions.items():
        words_list = [word for word in words_list if len(word) >= position and word[position-1] == letter]
        ## OK
        print("letter (lock): ", letter)
        print("Words List Update: ", words_list[:5])
    return words_list

'''
def filter_words_by_excluded_positions(words_list, excluded_positions):
    """Filters out words that have the specified letters in the excluded positions."""
    for position, letter in excluded_positions.items():
        words_list = [word for word in words_list if len(word) < position or word[position-1] != letter]
    return words_list
'''

def filter_words_by_excluded_positions(words_list, excluded_positions):
    """Filters out words that have the specified letters in the excluded positions."""
    filtered_words = []
    for word in words_list:
        exclude_word = False
        for position, letter in excluded_positions.items():
            if word[position-1] == letter:
                exclude_word = True
                break
        if not exclude_word:
            filtered_words.append(word)
    return filtered_words

def exclude_words_with_letters(words_list, excluded_letters):
    """Excludes words that contain any of the letters marked as incorrect by the user."""
    for letter in excluded_letters:
        words_list = [word for word in words_list if letter not in word]
        # ## OK
        # print("letter: ", letter)
        # print("Words List Update: ", words_list[:5])
    return words_list

def filter_words_by_wrong_positions(words_list, excluded_letters):
    # print('\nNew function, these letter must be included')
    # print('Wordlist before ', words_list[:10])
    # print('excluded_letters: ',excluded_letters)

    for _, letter in excluded_letters.items():
         words_list =  [word for word in words_list if letter in word]
    print('Wordlist after :', words_list[:10])
    return words_list

def filter_words_based_on_parameters_updated(words_list, letters_included, letters_not_included, locked_positions, excluded_positions):
    """Filters the word list based on the specified parameters."""
    print('Filter_words_based_on_parameters_updated Started')

    # Filter by letters not included
    words_list = exclude_words_with_letters(words_list, letters_not_included)

    # Filter by locked positions
    words_list = filter_words_by_locked_positions(words_list, locked_positions)
   
    # Filter by excluded positions
    words_list = filter_words_by_excluded_positions(words_list, excluded_positions)
    
    # Filter words list must inclue "Wrong Place" words
    words_list = filter_words_by_wrong_positions(words_list, excluded_positions)

    print('Filter_words_based_on_parameters_updated Ended')

    return words_list

def filter_word_frequency_dict(word_frequency_dict, words_list):
    filtered_dict = {}
    for word in words_list:
        if word in word_frequency_dict:
            filtered_dict[word] = word_frequency_dict[word]

    # Sort the dictionary by values (frequencies) in descending order
    sorted_filtered_dict = dict(sorted(filtered_dict.items(), key=lambda item: item[1], reverse=True))
    
    return sorted_filtered_dict or {}



## Recomendations for next word, Scoring

In [199]:
def most_common_combinations(words_list, starting_letters, top_n=5):
    """Finds the most common two-letter combinations in the given list of words."""
    two_letter_combinations = Counter()
    for word in words_list:
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                if word[i] in starting_letters and word[j] in starting_letters:
                    combo = tuple(sorted([word[i], word[j]]))
                    two_letter_combinations[combo] += 1
    return two_letter_combinations.most_common(top_n)

def compute_letter_scores(words_list):
    """Computes the frequency scores for each letter in the words list."""
    letter_frequency = Counter("".join(words_list))
    total_letters = sum(letter_frequency.values())
    
    # Normalize the letter scores
    letter_scores = {letter: freq / total_letters for letter, freq in letter_frequency.items()}
    
    return letter_scores

def compute_unique_letter_scores(words_list):
    """Computes and returns a dictionary with words as keys and their unique letter scores as values, sorted by highest score."""
    
    # Count the frequency of each letter in the entire words_list
    letter_frequency = Counter("".join(words_list))
    
    # For each word, sum the scores of its unique letters
    word_scores = {word: sum([letter_frequency[letter] for letter in set(word)]) for word in words_list}
    
    # Sort the dictionary by score in descending order
    sorted_word_scores = dict(sorted(word_scores.items(), key=lambda item: item[1], reverse=True))

    # Print the top 10 words and their scores
    print("\nTop 10 Unique letter score ")
    for i, (word, score) in enumerate(sorted_word_scores.items()):
        if i == 10:
            break
        print(f"{word}: {score}")
    return sorted_word_scores  

def compute_combination_scores(words_list):
    """Computes the scores for two-letter combinations in the words list."""
    two_letter_combinations = most_common_combinations(words_list,unique_letters, top_n=None)
    total_combinations = sum([count for _, count in two_letter_combinations])
    
    # Normalize the combination scores
    combination_scores = {combo: count / total_combinations for combo, count in two_letter_combinations}
    return combination_scores

def top_n_recommended_words_with_scores(words_list, n=20, repeat_multiplier=0.5):
    print('top_n_recommended_words_with_scores STARTED')
    top_common_letters = count_letter_frequency(words_list) ##!!
    """Recommends the top n optimal words based on letter and combination scores and returns the breakdown."""
    letter_scores = compute_letter_scores(words_list)
    combination_scores = compute_combination_scores(words_list)
    unique_letters_score = compute_unique_letter_scores(words_list)

    repeat_multiplier = float(repeat_multiplier_entry.get()) #Sets the multiplyer via the GUI

    word_scores = {}
    word_letter_scores = {}
    word_combination_scores = {}
    for word in words_list:
        # Letter score
        word_letter_score = sum([letter_scores.get(letter, 0) for letter in word])
        word_letter_scores[word] = round(word_letter_score, 2)
        
        # Combination score
        word_combination_score = 0
        for i in range(len(word)):
            for j in range(i+1, len(word)):
                combo = tuple(sorted([word[i], word[j]]))
                word_combination_score += combination_scores.get(combo, 0)
        word_combination_scores[word] = round(word_combination_score, 2)
        
        # Apply the multiplier if the word has repeated letters
        multiplier = repeat_multiplier if len(word) > len(set(word)) else 1.0
                
        # Total score
        word_scores[word] = round((word_letter_score * multiplier) + word_combination_score, 2)
    
    # Get the top n words with the highest scores
    n=20
    sorted_words = sorted(word_scores, key=word_scores.get, reverse=True)[:n]

    return sorted_words, [word_scores[word] for word in sorted_words], [word_letter_scores[word] for word in sorted_words], [word_combination_scores[word] for word in sorted_words], unique_letters_score


## Graphical interphase

In [200]:
# Command functions for buttons

def create_lock_command(lock_btn, exclude_btn, wrong_place_btn, position, letters_states):
    def command():
        letters_states[position]['locked'] = True
        letters_states[position]['excluded'] = False
        letters_states[position]['wrong_place'] = False

        lock_btn.config(bg="green")
        exclude_btn.config(bg="SystemButtonFace")
        wrong_place_btn.config(bg="SystemButtonFace")
    return command

def create_exclude_command(lock_btn, exclude_btn, wrong_place_btn, position, letters_states):
    def command():
        letters_states[position]['locked'] = False
        letters_states[position]['excluded'] = True
        letters_states[position]['wrong_place'] = False

        lock_btn.config(bg="SystemButtonFace")
        exclude_btn.config(bg="red")
        wrong_place_btn.config(bg="SystemButtonFace")
    return command

def create_wrong_place_command(lock_btn, exclude_btn, wrong_place_btn, position, letters_states):
    def command():
        letters_states[position]['locked'] = False
        letters_states[position]['excluded'] = False
        letters_states[position]['wrong_place'] = True

        lock_btn.config(bg="SystemButtonFace")
        exclude_btn.config(bg="SystemButtonFace")
        wrong_place_btn.config(bg="yellow")
    return command

def display_letters():

    global submitted_word  # Declare it as global
    submitted_word = word_entry.get().upper()  # Retrieve the word from the entry widget
    solve_wordle()
    # Initialize the local state for each letter
    letters_states = [{} for _ in submitted_word]

    for widget in root.winfo_children():
        widget.grid_forget()  # Clear the previous widgets

    # Initialize the local state for each letter
    letters_states = {i: {'locked': False, 'excluded': False, 'wrong_place': False} for i in range(5)}
    
    def solve_wordle_local():
        print('Solve Word Button pressed')
        global locked_positions, excluded_positions, letters_not_included, current_word_list ##!!

        locked_positions = {}
        excluded_positions = {}
        letters_not_included = []

        for position, state in letters_states.items():
            if state['locked']:
                locked_positions[position+1] = submitted_word[position]
            elif state['excluded']:
                letters_not_included.append(submitted_word[position])
            elif state['wrong_place']:
                excluded_positions[position+1] = submitted_word[position]

        temp = solve_wordle() ##!!
        ##print("temp: ", temp) ##!!

        current_word_list = filter_words_based_on_parameters_updated(
        current_word_list, 
        letters_included, 
        letters_not_included, 
        locked_positions, 
        excluded_positions)

        return temp ##!!

    solve_btn.config(command=solve_wordle_local)


    # Create all buttons first
    lock_buttons = []
    exclude_buttons = []
    wrong_place_buttons = []

    for i, letter in enumerate(submitted_word):
        ttk.Label(root, text=letter, width=5).grid(row=1, column=i+1)

        lock_button = tk.Button(root, text="Lock")
        lock_button.grid(row=2, column=i+1)
        lock_buttons.append(lock_button)

        exclude_button = tk.Button(root, text="Exclude")
        exclude_button.grid(row=3, column=i+1)
        exclude_buttons.append(exclude_button)

        wrong_place_button = tk.Button(root, text="Wrong Place")
        wrong_place_button.grid(row=4, column=i+1)
        wrong_place_buttons.append(wrong_place_button)

    # Now configure their commands
    for i, letter in enumerate(submitted_word):
        lock_buttons[i].config(command=create_lock_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], i, letters_states))
        exclude_buttons[i].config(command=create_exclude_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], i, letters_states))
        wrong_place_buttons[i].config(command=create_wrong_place_command(lock_buttons[i], exclude_buttons[i], wrong_place_buttons[i], i, letters_states))



    word_entry.grid(row=0, column=1, padx=10, pady=10)
    submit_btn.grid(row=0, column=2, padx=10, pady=10)
    solve_btn.grid(row=0, column=3, pady=10)
    #old_btn.grid(row=0, column=4, columnspan=5, pady=10)
    output_text.grid(row=7, column=0, columnspan=8, padx=10, pady=10)
    scrollbar.grid(row=7, column=6, sticky='ns')
    

def solve_wordle():
    global letters_included, letters_not_included, locked_positions, excluded_positions, filtered_words_updated, word_frequency_dict
    print('solve_wordle Started')
    word = submitted_word
    
    # Get the word from the entry
    word = submitted_word.upper()
    
    # Filter the words
    filtered_words_updated = filter_words_based_on_parameters_updated(current_word_list, letters_included, letters_not_included, locked_positions, excluded_positions)
    
    # Update word_frequency_dict
    word_frequency_dict = filter_word_frequency_dict(word_frequency_dict, filtered_words_updated)
        
    # Recommend the next word
    top_words, top_total_scores, top_letter_scores, top_combination_scores, unique_letter_scores = top_n_recommended_words_with_scores(filtered_words_updated, top_common_letters, unique_letters)

    # Top Commom letters in word_frequency_dict
    word_frequency_list = list(word_frequency_dict.keys())
    letter_frequency_in_word_frequency = count_letter_frequency(word_frequency_list)


    ###!!! Frequency Score
    letter_scores = compute_letter_scores(word_frequency_dict)
    print("### ", letter_scores)
    word_scores = calculate_word_scores(word_frequency_dict, letter_scores)
    print("Word scores :", word_scores)

    # Prepare the output to be displayed
    output_content = []

    # Extend output_content with only the top 20 items from word_frequency_filtered
    output_content.extend([f"{word} Freq: {freq}, Score: {word_scores.get(word, 'N/A')}" for word, freq in list(word_frequency_dict.items())[:20]])

    output_content.append((" "))
    output_content.extend([f"{letter} {freq}" for letter, freq in letter_frequency_in_word_frequency[:10]])
    ### Next step i s to exlude letter here tah are alreaddy locked in

    output_text.delete(1.0, tk.END)
    output_text.insert(tk.END, '\n'.join(output_content))

    return filtered_words_updated
    


In [201]:
# Load all possible words from git 

five_letter_words = fetch_gist_content("https://gist.github.com/dracos/dd0668f281e685bad51479e5acaadb93")
print('Five_letter_words collected. Len ',len(five_letter_words))

oldwords = get_old_words()
print('Old Words Collected. Len ', len(oldwords))

current_word_list = remove_words(five_letter_words, oldwords)
print('current_word_list. Len ', len(current_word_list))

## I NY old_word are not removed
current_word_list = keep_old_words(five_letter_words)
print("current_word_list reset. Len ", len(current_word_list))

word_frequency_dict = get_frequency_list()
print("Get word_frequency_dict completed")

top_common_letters = count_letter_frequency(current_word_list)
filtered_words, unique_letters = filter_words_based_on_letters(current_word_list, top_common_letters)

# Initialize the main window
root = tk.Tk()
root.title("Wordle Solver")

# Variables to store user's choices
letters_included = []
letters_not_included = []
locked_positions = {}
excluded_positions = {}
submitted_word = "" 
letter_states = {}  # {position: "locked"/"excluded"/"wrong_place"}

# Button for Submit Word
submit_btn = tk.Button(root, text="Submit Word", command=display_letters)
submit_btn.grid(row=0, column=2, padx=10, pady=10)
print('Submit Word Button pressed')

# Button to Solve Wordle
solve_btn = tk.Button(root, text="Solve Wordle", command=solve_wordle)
solve_btn.grid(row=0, column=3, columnspan=5, pady=10)
print('Solve Word Button pressed')

# Button for Exluding Old Words
old_btn = tk.Button(root, text="Keep Old Words", command=display_letters)
old_btn.grid(row=1, column=2, padx=10, pady=10)
print('Keep Old Word pressed')


# Text widget to display multiple lines of output
output_text = Text(root, width=50, height=22)
output_text.grid(row=7, column=0, columnspan=6, padx=10, pady=10)
scrollbar = Scrollbar(root, command=output_text.yview)
scrollbar.grid(row=7, column=6, sticky='nsew')
output_text.config(yscrollcommand=scrollbar.set)

# Entry for the next word
word_entry = ttk.Entry(root, width=15)
word_entry.grid(row=0, column=1, padx=10, pady=10)
word_entry.insert(0, "AEROS") #Suggested starting word

# Create a label for clarity
repeat_multiplier_label = ttk.Label(root, text="Repeat Multiplier:")
repeat_multiplier_label.grid(row=6, column=3, padx=10, pady=10, sticky='e')  # Adjust the row and column as needed

# The actual entry
repeat_multiplier_entry = ttk.Entry(root, width=5)
repeat_multiplier_entry.grid(row=6, column=4, padx=10, pady=10)
repeat_multiplier_entry.insert(0, "0.5")  # Default value

root.mainloop()


Five_letter_words collected. Len  14855
Old Words Collected. Len  870
current_word_list. Len  13985
Len word_list  14855
current_word_list reset. Len  14855
Get word_frequency_dict completed
Submit Word Button pressed
Solve Word Button pressed
Keep Old Word pressed
solve_wordle Started
Filter_words_based_on_parameters_updated Started
Wordlist after : ['AAHED', 'AALII', 'AAPAS', 'AARGH', 'AARTI', 'ABACA', 'ABACI', 'ABACK', 'ABACS', 'ABAFT']
Filter_words_based_on_parameters_updated Ended
top_n_recommended_words_with_scores STARTED

Top 10 Unique letter score 
AEROS: 31828
AROSE: 31828
SOARE: 31828
AESIR: 30997
ARISE: 30997
RAISE: 30997
REAIS: 30997
SERAI: 30997
SERIA: 30997
ALOES: 30894
###  {'A': 0.0964816084075851, 'B': 0.025885309572766736, 'O': 0.06917980351839159, 'U': 0.03506968243088874, 'T': 0.05421521590130226, 'H': 0.02773589216358236, 'E': 0.10253598355037698, 'R': 0.0669408270504912, 'W': 0.013228238519533928, 'I': 0.0584190084532785, 'C': 0.03456705506054375, 'F': 0.01617546

# Testing

In [202]:
def filter_words_by_excluded_positions_debug(words_list, excluded_positions):
    """Filters out words that have the specified letters in the excluded positions."""
    for position, letter in excluded_positions.items():
        print(f"Excluding letter {letter} from position {position}")
        words_list = [word for word in words_list if len(word) < position or word[position-1] != letter]
        print(f"Words list after filtering: {words_list}")
    return words_list


def test_filter_words_by_excluded_positions():
    words_list = ["BOOKS", "BOOST", "BOOTS", "BASKS", "BREAK"]
    excluded_positions = {1: 'O', 4: 'O', 5: 'O'}
    filtered_words = filter_words_by_excluded_positions(words_list, excluded_positions)
    assert filtered_words == ["BASKS", "BREAK"], f"Expected ['BASKS', 'BREAK'], but got {filtered_words}"
    print(f"Test 1 passed!")

# test_filter_words_by_excluded_positions()





In [203]:
import unittest

def test_filter_words_by_excluded_positions():
  """Tests the filter_words_by_excluded_positions() function."""

  # Test data
  words_list = ["hello", "world", "apple", "banana", "orange"]
  excluded_positions = {2: "l"}

  # Expected result
  expected_filtered_words = ["world", "apple", "orange"]

  # Actual result
  actual_filtered_words = filter_words_by_excluded_positions(words_list, excluded_positions)

  # Assert that the expected result matches the actual result
  self.assertEqual(expected_filtered_words, actual_filtered_words)


In [204]:
# import unittest

# def test_filter_words_by_excluded_positions():
#   """Tests the filter_words_by_excluded_positions() function."""

#   # Test data
#   words_list = ["hello", "world", "apple", "banana", "orange"]
#   excluded_positions = {2: "l"}

#   # Expected result
#   expected_filtered_words = ["world", "apple", "orange"]

#   # Actual result
#   actual_filtered_words = filter_words_by_excluded_positions(words_list, excluded_positions)

#   # Assert that the expected result matches the actual result
#   self.assertEqual(expected_filtered_words, actual_filtered_words)

# if __name__ == '__main__':
#     unittest.main()

In [205]:
temp_word_frequency_dict = get_frequency_list()
temp_word_frequency_list = list(temp_word_frequency_dict.keys())
print(temp_word_frequency_list[0:10])
temp_freq = count_letter_frequency(temp_word_frequency_list)
print(temp_freq)

temp_new_words = filter_words_based_on_letters(temp_word_frequency_list, temp_freq)
print(temp_new_words)

['ABOUT', 'OTHER', 'WHICH', 'THEIR', 'THERE', 'FIRST', 'WOULD', 'THESE', 'CLICK', 'PRICE']
[('A', 21942), ('E', 18907), ('O', 14627), ('I', 13749), ('S', 13683), ('R', 12185), ('N', 11447), ('L', 10375), ('T', 9856), ('C', 7422), ('D', 6970), ('M', 6957), ('U', 6806), ('H', 5633), ('G', 5584), ('P', 5511), ('B', 5003), ('K', 4966), ('Y', 4617), ('F', 3208), ('V', 2597), ('W', 2391), ('Z', 1868), ('J', 1485), ('X', 1456), ('Q', 420)]
(['AERIO', 'EOVIA', 'AOIFE', 'AEIOU', 'EOLIA', 'OAKIE', 'EMAIO'], ['A', 'I', 'V', 'E', 'R', 'K', 'L', 'M', 'U', 'F', 'O'])


In [206]:
temp_top_common_letters = count_letter_frequency(temp_word_frequency_list)
temp_filtered_words, unique_letters = filter_words_based_on_letters(temp_word_frequency_list, temp_top_common_letters)
print(temp_filtered_words)

temp = get_most_common_letter_positions(temp_filtered_words, unique_letters)
print(temp)

['AERIO', 'EOVIA', 'AOIFE', 'AEIOU', 'EOLIA', 'OAKIE', 'EMAIO']
[('A', 0), ('E', 0), ('M', 1), ('O', 1), ('V', 2), ('R', 2), ('K', 2), ('L', 2), ('I', 3), ('F', 3), ('U', 4)]


In [207]:
word_list = ['apple', 'apply', 'banana', 'grape', 'peach', 'pear']
letters = ['p', 'a']
print(get_most_common_letter_positions(word_list, letters))


[('p', 0), ('a', 2)]


In [208]:
# Sample input for testing
word_frequency_dict_sample = {
    "BOOKS": 100,
    "BOOST": 90,
    "BOOTS": 80,
    "BASKS": 70,
    "BREAK": 60
}
letter_frequency_in_word_frequency_sample = ['B', 'O', 'A', 'K', 'S', 'T', 'E', 'R']

temp_word_frequency_list = letter_frequency = Counter("".join(words_list))

print(count_letter_frequency(temp_word_frequency_list))

# Testing the function

print(calculate_word_scores(word_frequency_dict_sample, letter_frequency_in_word_frequency_sample))

NameError: name 'words_list' is not defined

In [ ]:
# Initialize an empty dictionary to store letter frequencies for each word
letter_frequencies_dict = {}

# Iterate through the keys (words) in the sample dictionary
for word in word_frequency_dict_sample.keys():
    # Calculate the letter frequencies for the current word
    letter_frequency = Counter(word)
    
    # Add the letter frequencies to the new dictionary
    letter_frequencies_dict[word] = letter_frequency

# Print the resulting dictionary of letter frequencies for each word
for word, letter_frequency in letter_frequencies_dict.items():
    print(f"Word: {word}")
    print(f"Letter Frequencies: {letter_frequency}")
    print()


Word: BOOKS
Letter Frequencies: Counter({'O': 2, 'B': 1, 'K': 1, 'S': 1})

Word: BOOST
Letter Frequencies: Counter({'O': 2, 'B': 1, 'S': 1, 'T': 1})

Word: BOOTS
Letter Frequencies: Counter({'O': 2, 'B': 1, 'T': 1, 'S': 1})

Word: BASKS
Letter Frequencies: Counter({'S': 2, 'B': 1, 'A': 1, 'K': 1})

Word: BREAK
Letter Frequencies: Counter({'B': 1, 'R': 1, 'E': 1, 'A': 1, 'K': 1})



In [ ]:
# Initialize a Counter to keep track of letter frequencies
all_letters_frequency = Counter()

# Iterate through the keys (words) in the sample dictionary
for word in word_frequency_dict_sample.keys():
    # Update the all_letters_frequency Counter with the letter frequencies for the current word
    all_letters_frequency.update(word)

# Sort the letter frequencies in descending order
sorted_letter_frequencies = all_letters_frequency.most_common()

# Print the sorted letter frequencies
for letter, frequency in sorted_letter_frequencies:
    print(f"Letter: {letter}, Frequency: {frequency}")

Letter: O, Frequency: 6
Letter: B, Frequency: 5
Letter: S, Frequency: 5
Letter: K, Frequency: 3
Letter: T, Frequency: 2
Letter: A, Frequency: 2
Letter: R, Frequency: 1
Letter: E, Frequency: 1
